IMPORTING LIBRARIES

In [59]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

In [60]:
csv_file_path = '/predict.csv'
df = pd.read_csv(csv_file_path)

In [61]:
df.shape

(4956, 27)

In [62]:
df['date'] = pd.to_datetime(df['date'])

<ipython-input-62-e8d2d516eb0e>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['date'] = pd.to_datetime(df['date'])


In [63]:
df = df.dropna(subset=['time'])
df = df.dropna(subset=['dist'])

In [64]:
df['venue_code'] = df['venue'].astype('category').cat.codes
df['opponent_code'] = df['opponent'].astype('category').cat.codes
df['hour'] = df['time'].str.replace(':.+','', regex=True).astype('int')
df['day_code'] = df['date'].dt.dayofweek
df['target'] = (df["result"] == "W").astype('int')

In [65]:
from sklearn.ensemble import RandomForestClassifier

In [66]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [67]:
train = df[df['date']< '2023-01-01']

In [68]:
test = df[df['date']> '2023-01-01']

In [69]:
predictors = ['venue_code','opponent_code','hour','day_code']

In [70]:
rf.fit(train[predictors],train['target'])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [71]:
preds = rf.predict(test[predictors])

In [72]:
from sklearn.metrics import accuracy_score

In [73]:
acc = accuracy_score(test['target'], preds)

In [74]:
acc

0.5728155339805825

In [75]:
combined = pd.DataFrame(dict(actual = test['target'], predictions = preds))

In [76]:
pd.crosstab(index=combined['actual'], columns = combined['predictions'])

predictions,0,1
actual,,
0,369,129
1,223,103


In [77]:
from sklearn.metrics import precision_score

In [78]:
precision_score(test['target'], preds)

0.44396551724137934

In [79]:
df = df.dropna(subset=['time'])
df = df.dropna(subset=['date'])


In [80]:
grouped_matches = df.groupby('team')

In [81]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values('date')
    rolling_stats = group[cols].rolling(3, closed = 'left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group


In [82]:
cols = ['gf','ga','sh','sot','dist','fk','pk','pkatt','xg','xga']

new_cols = [f'{c}_rolling' for c in cols]

In [83]:
# returns 0 columns when trying to sort by date before calculating rolling averages

In [84]:
matches_rolling = df.groupby('team').apply(lambda x: rolling_averages(x, cols, new_cols))

In [85]:
matches_rolling

date   time            comp         round  \
team                                                                           
Arsenal                 4387 2017-08-19  17:30  Premier League   Matchweek 2   
                        4388 2017-08-27  16:00  Premier League   Matchweek 3   
                        4389 2017-09-09  15:00  Premier League   Matchweek 4   
                        4390 2017-09-17  13:30  Premier League   Matchweek 5   
                        4391 2017-09-25  20:00  Premier League   Matchweek 6   
...                                 ...    ...             ...           ...   
Wolverhampton Wanderers 878  2023-12-03  16:30  Premier League  Matchweek 27   
                        215  2023-12-17  14:00  Premier League  Matchweek 17   
                        216  2023-12-24  13:00  Premier League  Matchweek 18   
                        217  2023-12-27  19:30  Premier League  Matchweek 19   
                        218  2023-12-30  15:00  Premier League  Matchweek 20   

                              day venue result  gf  ga       opponent  ...  \
team                                                                   ...   
Arsenal                 4387  Sat  Away      L   0   1     Stoke City  ...   
                        4388  Sun  Away      L   0   4      Liverpool  ...   
                        4389  Sat  Home      W   3   0    Bournemouth  ...   
                        4390  Sun  Away      D   0   0        Chelsea  ...   
                        4391  Mon  Home      W   2   0      West Brom  ...   
...                           ...   ...    ...  ..  ..            ...  ...   
Wolverhampton Wanderers 878   Sun  Away      L   1   2  Newcastle Utd  ...   
                        215   Sun  Away      L   0   3       West Ham  ...   
                        216   Sun  Home      W   2   1        Chelsea  ...   
                        217   Wed  Away      W   4   1      Brentford  ...   
                        218   Sat  Home      W   3   0        Everton  ...   

                              gf_rolling  ga_rolling  sh_rolling  sot_rolling  \
team                                                                            
Arsenal                 4387    1.333333    2.000000   21.333333     8.333333   
                        4388    0.666667    2.333333   19.000000     7.666667   
                        4389    0.333333    2.666667   10.666667     3.000000   
                        4390    1.000000    1.666667   14.333333     5.000000   
                        4391    1.000000    1.333333   12.000000     3.666667   
...                                  ...         ...         ...          ...   
Wolverhampton Wanderers 878     2.000000    1.666667   12.333333     3.333333   
                        215     1.666667    2.000000   11.000000     4.333333   
                        216     1.000000    2.666667   10.000000     4.000000   
                        217     1.000000    2.000000   11.666667     4.333333   
                        218     2.000000    1.666667   13.000000     4.666667   

                             dist_rolling fk_rolling pk_rolling pkatt_rolling  \
team                                                                            
Arsenal                 4387    18.533333   1.666667   0.000000      0.000000   
                        4388    18.433333   1.000000   0.000000      0.000000   
                        4389    18.400000   0.333333   0.000000      0.000000   
                        4390    16.766667   0.333333   0.000000      0.000000   
                        4391    16.566667   0.333333   0.000000      0.000000   
...                                   ...        ...        ...           ...   
Wolverhampton Wanderers 878     17.933333   0.000000   0.333333      0.333333   
                        215     18.200000   0.333333   0.333333      0.333333   
                        216     18.866667   0.333333   0.333333      0.333333   
                        217     18

In [86]:
matches_rolling.index = range(matches_rolling.shape[0])

In [87]:
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling,xg_rolling,xga_rolling
0,2017-08-19,17:30,Premier League,Matchweek 2,Sat,Away,L,0,1,Stoke City,...,1.333333,2.000000,21.333333,8.333333,18.533333,1.666667,0.000000,0.000000,2.466667,1.333333
1,2017-08-27,16:00,Premier League,Matchweek 3,Sun,Away,L,0,4,Liverpool,...,0.666667,2.333333,19.000000,7.666667,18.433333,1.000000,0.000000,0.000000,2.166667,1.433333
2,2017-09-09,15:00,Premier League,Matchweek 4,Sat,Home,W,3,0,Bournemouth,...,0.333333,2.666667,10.666667,3.000000,18.400000,0.333333,0.000000,0.000000,0.800000,1.866667
3,2017-09-17,13:30,Premier League,Matchweek 5,Sun,Away,D,0,0,Chelsea,...,1.000000,1.666667,14.333333,5.000000,16.766667,0.333333,0.000000,0.000000,1.433333,1.466667
4,2017-09-25,20:00,Premier League,Matchweek 6,Mon,Home,W,2,0,West Brom,...,1.000000,1.333333,12.000000,3.666667,16.566667,0.333333,0.000000,0.000000,1.400000,1.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4858,2023-12-03,16:30,Premier League,Matchweek 27,Sun,Away,L,1,2,Newcastle Utd,...,2.000000,1.666667,12.333333,3.333333,17.933333,0.000000,0.333333,0.333333,1.433333,1.566667
4859,2023-12-17,14:00,Premier League,Matchweek 17,Sun,Away,L,0,3,West Ham,...,1.666667,2.000000,11.000000,4.333333,18.200000,0.333333,0.333333,0.333333,1.533333,1.800000
4860,2023-12-24,13:00,Premier League,Matchweek 18,Sun,Home,W,2,1,Chelsea,...,1.000000,2.666667,10.000000,4.000000,18.866667,0.333333,0.333333,0.333333,1.066667,1.933333
4861,2023-12-27,19:30,Premier League,Matchweek 19,Wed,Away,W,4,1,Brentford,...,1.000000,2.000000,11.666667,4.333333,18.900000,0.666667,0.000000,0.000000,1.033333,1.633333


In [88]:
predictors = ['venue_code', 'opponent_code', 'hour', 'day_code', 'gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling','xg','xga']

In [89]:
def make_predictions(data, predictors):
    train = matches_rolling[matches_rolling['date']< '2022-03-01']
    test = matches_rolling[matches_rolling['date']> '2022-03-01']
    rf.fit(train[predictors],train['target'])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test['target'], predicted = preds), index = test.index)
    precision = precision_score(test['target'], preds)
    return combined, precision

In [90]:
combined, precision = make_predictions(matches_rolling, predictors)

In [91]:
precision

0.693446088794926

In [92]:
combined

,actual,predicted
174,1,1
175,0,1
176,1,1
177,0,0
178,0,0
...,...,...
4858,0,0
4859,0,0
4860,1,0
4861,1,1


In [93]:
combined = combined.merge(matches_rolling[['date','team','opponent', 'result']], left_index = True, right_index = True)

In [94]:
combined

,actual,predicted,date,team,opponent,result
174,1,1,2022-03-13,Arsenal,Leicester City,W
175,0,1,2022-03-16,Arsenal,Liverpool,L
176,1,1,2022-03-19,Arsenal,Aston Villa,W
177,0,0,2022-04-04,Arsenal,Crystal Palace,L
178,0,0,2022-04-09,Arsenal,Manchester Utd,L
...,...,...,...,...,...,...
4858,0,0,2023-12-03,Wolverhampton Wanderers,Newcastle Utd,L
4859,0,0,2023-12-17,Wolverhampton Wanderers,West Ham,L
4860,1,0,2023-12-24,Wolverhampton Wanderers,Chelsea,W
4861,1,1,2023-12-27,Wolverhampton Wanderers,Brentford,W


In [95]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {
    'Brighton and Hove Albion': 'Brighton',
    'Manchester United': 'Manchester Utd',
    'Huddersfield Town': 'Huddersfield',
    'Newcastle United': "Newcastle Utd",
    'Nottingham Forest': "Nott'ham Forest",
    'Sheffield United': 'Sheffield Utd',
    'Tottenham Hotspur': 'Tottenham',
    'West Bromwich': 'West Brom',
    'West Ham United': 'West Ham',
    'Wolverhampton Wanderers': 'Wolves'

}

mapping = MissingDict(**map_values)

In [96]:
mapping['West Ham United']

'West Ham'

In [97]:
combined['new_team'] = combined['team'].map(mapping)

In [98]:
combined

,actual,predicted,date,team,opponent,result,new_team
174,1,1,2022-03-13,Arsenal,Leicester City,W,Arsenal
175,0,1,2022-03-16,Arsenal,Liverpool,L,Arsenal
176,1,1,2022-03-19,Arsenal,Aston Villa,W,Arsenal
177,0,0,2022-04-04,Arsenal,Crystal Palace,L,Arsenal
178,0,0,2022-04-09,Arsenal,Manchester Utd,L,Arsenal
...,...,...,...,...,...,...,...
4858,0,0,2023-12-03,Wolverhampton Wanderers,Newcastle Utd,L,Wolves
4859,0,0,2023-12-17,Wolverhampton Wanderers,West Ham,L,Wolves
4860,1,0,2023-12-24,Wolverhampton Wanderers,Chelsea,W,Wolves
4861,1,1,2023-12-27,Wolverhampton Wanderers,Brentford,W,Wolves


In [99]:
merged = combined.merge(combined, left_on=['date','new_team'], right_on = ['date', 'opponent'])

In [100]:
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,1,1,2022-03-13,Arsenal,Leicester City,W,Arsenal,0,0,Leicester City,Arsenal,L,Leicester City
1,0,1,2022-03-16,Arsenal,Liverpool,L,Arsenal,1,1,Liverpool,Arsenal,W,Liverpool
2,1,1,2022-03-19,Arsenal,Aston Villa,W,Arsenal,0,0,Aston Villa,Arsenal,L,Aston Villa
3,0,0,2022-04-04,Arsenal,Crystal Palace,L,Arsenal,1,0,Crystal Palace,Arsenal,W,Crystal Palace
4,0,0,2022-04-09,Arsenal,Manchester Utd,L,Arsenal,1,0,Manchester United,Arsenal,W,Manchester Utd
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,0,0,2023-12-03,Wolverhampton Wanderers,Newcastle Utd,L,Wolves,1,0,Newcastle United,Wolves,W,Newcastle Utd
1378,0,0,2023-12-17,Wolverhampton Wanderers,West Ham,L,Wolves,1,0,West Ham United,Wolves,W,West Ham
1379,1,0,2023-12-24,Wolverhampton Wanderers,Chelsea,W,Wolves,0,0,Chelsea,Wolves,L,Chelsea
1380,1,1,2023-12-27,Wolverhampton Wanderers,Brentford,W,Wolves,0,0,Brentford,Wolves,L,Brentford


In [101]:
merged[(merged['predicted_x'] == 1) & (merged['predicted_y'] == 0 )]['actual_x'].value_counts()

1    326
0    140
Name: actual_x, dtype: int64

In [102]:
326/466

0.6995708154506438